In [1]:
#Импорт библиотек
import os
import mat_properties as prop
import numpy as n
import pandas as pd
import cotel
import nasos
import time
from scipy.optimize import root

#Время начала расчёта


#Основные констант3
calctolerance=10**-3
calcmethod = "hybr"
KPDnasos = 0.8074
KPD=0.99

#таблица номинального режима
gas_streams0   = pd.read_excel('streams0.xlsx', sheet_name='gas',  index_col=0)
water_streams0 = pd.read_excel('streams0.xlsx', sheet_name='water',index_col=0)
#таблица задаваемых значений (заменится на интерфейс)
gas_streamsZAD    = pd.read_excel('streams.xlsx',  sheet_name='gas',  index_col=0)
water_streamsZAD  = pd.read_excel('streams.xlsx',  sheet_name='water',index_col=0)
#рабочая таблица (=номинал в 1 итерации)
gas_streams = gas_streams0.copy()
water_streams = water_streams0.copy()
#рабочая таблица показателей блоков
heaters  = pd.read_excel('blocks.xlsx',  sheet_name='heaters',index_col=0)
electric  = pd.read_excel('blocks.xlsx',  sheet_name='electric',index_col=0)


#чтение с вводного экселя входных данных
Tgas = gas_streamsZAD.loc['GTU-PEVD','T']
Pgas = gas_streamsZAD.loc['GTU-PEVD','P']
Ggas = gas_streamsZAD.loc['GTU-PEVD','G']
Propgas = gas_streamsZAD.loc['GTU-PEVD','N2':'Ar']
Pvd = water_streamsZAD.loc['IVD-PEVD','P']
Pnd = water_streamsZAD.loc['IND-PPND','P']
Twat0 = water_streamsZAD.loc['X-GPK','T']

#запись состава газов в раб табл
gas_streams['N2'] = Propgas[0]
gas_streams['O2'] = Propgas[1]
gas_streams['CO2'] = Propgas[2]
gas_streams['H2O'] = Propgas[3]
gas_streams['Ar'] = Propgas[4]

#Состав газов в номинальном режиме и в частичной нагрузке
gasmix = "Nitrogen*Oxygen*CO2*Water*Argon"
fractiongas0 = (gas_streams0.at['GTU-PEVD','N2'],gas_streams0.at['GTU-PEVD','O2'],gas_streams0.at['GTU-PEVD','CO2'],gas_streams0.at['GTU-PEVD','H2O'],gas_streams0.at['GTU-PEVD','Ar'])
fractiongas  = (gas_streams.at['GTU-PEVD','N2'],gas_streams.at['GTU-PEVD','O2'],gas_streams.at['GTU-PEVD','CO2'],gas_streams.at['GTU-PEVD','H2O'],gas_streams.at['GTU-PEVD','Ar'])

#Считывание рефпропа
RP = prop.init_REFPROP(r'C:\Program Files (x86)\REFPROP')
os.environ['RPPREFIX'] = r'C:/Program Files (x86)/REFPROP'
gas1 = prop.Materials_prop(gasmix,fractiongas, prop.REFPROP_h_s,prop.REFPROP_p_t,prop.REFPROP_p_h,prop.REFPROP_p_s,prop.REFPROP_p_q,prop.REFPROP_t_q,RP=RP)
gas0 = prop.Materials_prop(gasmix,fractiongas0, prop.REFPROP_h_s,prop.REFPROP_p_t,prop.REFPROP_p_h,prop.REFPROP_p_s,prop.REFPROP_p_q,prop.REFPROP_t_q,RP=RP)
water=prop.Materials_prop('water',[1.0,0,0,0,0], prop.REFPROP_h_s,prop.REFPROP_p_t,prop.REFPROP_p_h,prop.REFPROP_p_s,prop.REFPROP_p_q,prop.REFPROP_t_q,RP=RP)

#Задание в раб табл параметров газа (1 строка)
gas_streams.loc['GTU-PEVD','T'] = Tgas
gas_streams['G'] = Ggas
gas_streams['P'] = Pgas
gas_streams.loc['GTU-PEVD','H'] = gas1.p_t(Pgas,Tgas)['h']

#Задание энтальпий газа в номинальном режиме
Temperatures= gas_streams0.loc['GTU-PEVD':'GPK-out','T']
Enthalpies=list(map( lambda x: gas1.p_t(Pgas,x)['h'],Temperatures))
gas_streams0.loc['GTU-PEVD':'GPK-out','H'] = Enthalpies

#Задание в раб табл параметров по воде
water_streams.loc['IVD-PEVD':'PEN-EVD','P'] = Pvd
water_streams.loc['IND-PPND':'X-GPK','P'] = Pnd
water_streams.loc['BND-PEN','P'] = Pnd
water_streams.loc['X-GPK','T'] = Twat0

#Расчёт параметров на насыщении в испарительных поверхностях (в 1 итерации приближение: недогрев до н.п. на входе 3°С)
water_streams.loc['IVD-PEVD','T'] = water.p_q(Pvd,1)['T']
water_streams.loc['IVD-PEVD','H'] = water.p_q(Pvd,1)['h']
water_streams.loc['IND-PPND','T'] = water.p_q(Pnd,1)['T']
water_streams.loc['IND-PPND','H'] = water.p_q(Pnd,1)['h']
water_streams.loc['EVD-IVD','T'] = water.p_q(Pvd,1)['T']-3
water_streams.loc['GPK-IND','T'] = water.p_q(Pnd,1)['T']-3
water_streams.loc['EVD-IVD','H'] = water.p_t(Pvd,water_streams.loc['EVD-IVD','T'])['h']
water_streams.loc['GPK-IND','H'] = water.p_t(Pnd,water_streams.loc['GPK-IND','T'])['h']

#Задача моделей

PEVD_obj = cotel.heatex('GTU-PEVD','PEVD-IVD','IVD-PEVD','PEVD-DROSVD',
                    KPD,calctolerance,gas0,gas1,water,calcmethod,gas_streams0,water_streams0,gas_streams,water_streams)

IVD_obj  = cotel.evaporVD ('PEVD-IVD','IVD-EVD','EVD-IVD','IVD-PEVD',          
                           KPD,calctolerance,gas1,gas0,water,calcmethod,gas_streams0,water_streams0,gas_streams,water_streams)

EVD_obj = cotel.heatex('IVD-EVD','EVD-PPND','PEN-EVD','EVD-IVD',               
                   KPD,calctolerance,gas0,gas1,water,calcmethod,gas_streams0,water_streams0,gas_streams,water_streams)


PEN_obj =nasos.nasos('BND-PEN','PEN-EVD',water,KPDnasos,water_streams)


PPND_obj = cotel.heatexPEND('EVD-PPND','PPND-IND','IND-PPND','PPND-DROSND',    
                            KPD,calctolerance,gas0,gas1,water,calcmethod,gas_streams0,water_streams0,gas_streams,water_streams)

IND_obj  = cotel.evaporND('PPND-IND','IND-GPK','GPK-IND','IND-PPND',  'PEVD-DROSVD',    
                             KPD,calctolerance,gas1,gas0,water,calcmethod,gas_streams0,water_streams0,gas_streams,water_streams)

GPK_obj = cotel.heatex('IND-GPK','GPK-out','X-GPK','GPK-IND',
                       KPD,calctolerance,gas1,gas0,water,calcmethod,gas_streams0,water_streams0,gas_streams,water_streams)
start_time = time.time()
#Расчёт
it=60
for i in range(it):
    ######################################################################
    #Связвка высокого давления
    for j in range(it):
        
        start_timePEVD = time.time()

        #Расчёт ПЕВД
        PEVD = PEVD_obj.calc()
        gas_streams.loc['PEVD-IVD','T':'G'] = [PEVD['Tg'],PEVD['Pg'],PEVD['Hg'],PEVD['Gg']]
        water_streams.loc['PEVD-DROSVD','T':'G'] = [PEVD['Tw'],PEVD['Pw'],PEVD['Hw'],PEVD['Gw']]
        
        print("PEVD:--- %s сек. ---" % round((time.time() - start_timePEVD),1))
        
        start_timeIVD = time.time()

        #Расчёт ИВД
        IVD  =       IVD_obj.calc()
        gas_streams.loc['IVD-EVD','T':'G'] = [IVD[0],IVD[1],IVD[2],IVD[3]]
        water_streams.loc['IVD-PEVD','T':'G'] = [IVD[4],IVD[5],IVD[6],IVD[7]]
        
        print("IVD:--- %s сек. ---" % round((time.time() - start_timeIVD),1))

        
        #Переопределение расхода в ВД
        water_streams.loc['PEVD-DROSVD':'PEN-EVD','G'] = IVD[7]
        water_streams.loc['BND-PEN','G'] = IVD[7]

        
        start_timeEVD = time.time()

        #Расчёт ЭВД
        EVD = EVD_obj.calc()
        print("EVD:--- %s сек. ---" % round((time.time() - start_timePEVD),1))

        gas_streams.loc['EVD-PPND','T':'G'] = [EVD['Tg'],EVD['Pg'],EVD['Hg'],EVD['Gg']]
        water_streams.loc['EVD-IVD','T':'G'] = [EVD['Tw'],EVD['Pw'],EVD['Hw'],EVD['Gw']]
        

        
        #Баланс ПЕВ+ИВД+ЭВД
        Qgas1 = KPD*gas_streams.at['GTU-PEVD','G']*(gas_streams.at['GTU-PEVD','H']-gas_streams.at['IVD-EVD','H'])
        Qwat1 = water_streams.at['PEVD-DROSVD','G']*(water_streams.at['PEVD-DROSVD','H']-water_streams.at['EVD-IVD','H'])
        print('dQ/Q ПЕВД+ИВД+ЭВД',((Qgas1-Qwat1)/Qgas1*100))
        if  abs((Qgas1-Qwat1)/Qgas1*100) < calctolerance:
            break;
    print("ВД: --- %s сек. ---" % round((time.time() - start_time),1))
    ######################################################################
    
    #Для сходимости
    if i==0:
        gas_streams.loc['PPND-IND','T'] = gas_streams.loc['EVD-PPND','T'] -1
        gas_streams.loc['PPND-IND','H'] = gas1.p_t(gas_streams.loc['PPND-IND','P'],gas_streams.loc['PPND-IND','T'])['h']
    
    ######################################################################
    #Связка низкого давления
    for j in range(it): 
        #Расчёт ППНД
        PPND = PPND_obj.calc()
        gas_streams.loc['PPND-IND','T':'G']    = [PPND[0],PPND[1],PPND[2],PPND[3]]
        water_streams.loc['PPND-DROSND','T':'G'] = [PPND[4],PPND[5],PPND[6],PPND[7]]
        
        #Расчёт ИНД
        IND=IND_obj.calc()
        gas_streams.loc['IND-GPK','T':'G']    = [IND[0],IND[1],IND[2],IND[3]]
        water_streams.loc['IND-PPND','T':'G'] = [IND[4],IND[5],IND[6],IND[7]]
        
        #Переопределение расхода в НД
        water_streams.loc['PPND-DROSND':'IND-PPND','G'] = IND[7]
        
        #ПЭН
        water_streams.loc['BND-PEN','T':'G'] = [IND[9],IND[10],IND[11],IND[12]]
        PEN=PEN_obj.calc()
        water_streams.loc['PEN-EVD','T':'G'] = [PEN[0],PEN[1],PEN[2],PEN[3]]
        
        #Баланс ППНД+ИНД
        Qgas = KPD*gas_streams.at['EVD-PPND','G']*(gas_streams.at['EVD-PPND','H']-gas_streams.at['IND-GPK','H'])
        Qwat = water_streams.at['IND-PPND','G']*(water_streams.at['IND-PPND','H']-water_streams.at['GPK-IND','H'])+\
        water_streams.at['PPND-DROSND','G']*(water_streams.at['PPND-DROSND','H']-\
        water_streams.at['IND-PPND','H'])+water_streams.at['BND-PEN','G']*(water_streams.at['BND-PEN','H']-water_streams.at['GPK-IND','H'])
        
        #Расчёт расхода в ГПК
        water_streams.loc['X-GPK','G'] = water_streams.at['PPND-DROSND','G']+water_streams.at['PEVD-DROSVD','G']
        water_streams.loc['GPK-IND','G'] = water_streams.at['X-GPK','G']
        
        
        start_timeGPK = time.time()
        #Расчёт ГПК
        GPK = GPK_obj.calc()
        gas_streams.loc['GPK-out','T':'G']   = [GPK['Tg'],GPK['Pg'],GPK['Hg'],GPK['Gg']]
        water_streams.loc['GPK-IND','T':'G'] = [GPK['Tw'],GPK['Pw'],GPK['Hw'],GPK['Gw']]
        print("GPK:--- %s сек. ---" % round((time.time() - start_timeGPK),1))
        
        
        #Баланс ППНД+ИНД+ГПК
        Qgas1 = KPD*gas_streams.at['EVD-PPND','G']*(gas_streams.at['EVD-PPND','H']-gas_streams.at['GPK-out','H'])
        Qwat1 = water_streams.at['GPK-IND','G']*(water_streams.at['GPK-IND','H']-water_streams.at['X-GPK','H'])+\
        water_streams.at['IND-PPND','G']*(water_streams.at['IND-PPND','H']-water_streams.at['GPK-IND','H'])+\
        water_streams.at['PPND-DROSND','G']*(water_streams.at['PPND-DROSND','H']-water_streams.at['IND-PPND','H'])+\
        water_streams.at['BND-PEN','G']*(water_streams.at['BND-PEN','H']-water_streams.at['GPK-IND','H'])
        print('dQ/Q ППНД+ИНД+ГПК',(Qgas1-Qwat1)/Qgas1*100)
        if  abs((Qgas1-Qwat1)/Qgas1*100) < calctolerance:
            break;
    print("НД+ --- %s сек. ---" % round((time.time() - start_time),1))
    ######################################################################

    #Баланс общий
    Qgasall = KPD*gas_streams.at['GTU-PEVD','G']*(gas_streams.at['GTU-PEVD','H']-gas_streams.at['GPK-out','H'])
    Qwatall = water_streams.at['PPND-DROSND','G']*(water_streams.at['PPND-DROSND','H']-water_streams.at['X-GPK','H'])+water_streams.at['PEVD-DROSVD','G']*(water_streams.at['PEVD-DROSVD','H']-water_streams.at['X-GPK','H'])-water_streams.at['BND-PEN','G']*(water_streams.at['PEN-EVD','H']-water_streams.at['BND-PEN','H'])
    print('dQ/Qsumm',(Qgasall-Qwatall)/Qgasall*100)
    if  abs((Qgasall-Qwatall)/Qgasall) < calctolerance:
        break;

print('Степень сухости пара в ЭВД: ',water.p_h(water_streams.at['EVD-IVD','P'],water_streams.at['EVD-IVD','H'])['Q'])
print('Степень сухости пара в ГПК: ',water.p_h(water_streams.at['GPK-IND','P'],water_streams.at['GPK-IND','H'])['Q'])
print("fin:--- %s сек. ---" % round((time.time() - start_time),1))

Prop:--- 0.0 сек. ---
Iter:--- 0.4 сек. ---
Else:--- 0.0 сек. ---
PEVD:--- 0.4 сек. ---
IVD:--- 0.6 сек. ---
Prop:--- 0.1 сек. ---
Iter:--- 0.5 сек. ---
Else:--- 0.0 сек. ---
EVD:--- 1.6 сек. ---
dQ/Q ПЕВД+ИВД+ЭВД -0.32731888771103046
Prop:--- 0.0 сек. ---
Iter:--- 0.4 сек. ---
Else:--- 0.0 сек. ---
PEVD:--- 0.4 сек. ---
IVD:--- 0.6 сек. ---
Prop:--- 0.0 сек. ---
Iter:--- 0.4 сек. ---
Else:--- 0.0 сек. ---
EVD:--- 1.5 сек. ---
dQ/Q ПЕВД+ИВД+ЭВД -0.5400697353503915
Prop:--- 0.0 сек. ---
Iter:--- 0.4 сек. ---
Else:--- 0.0 сек. ---
PEVD:--- 0.4 сек. ---
IVD:--- 0.6 сек. ---
Prop:--- 0.0 сек. ---
Iter:--- 0.4 сек. ---
Else:--- 0.0 сек. ---
EVD:--- 1.4 сек. ---
dQ/Q ПЕВД+ИВД+ЭВД -0.32564984924157375
Prop:--- 0.0 сек. ---
Iter:--- 0.4 сек. ---
Else:--- 0.0 сек. ---
PEVD:--- 0.4 сек. ---
IVD:--- 0.6 сек. ---
Prop:--- 0.0 сек. ---
Iter:--- 0.4 сек. ---
Else:--- 0.0 сек. ---
EVD:--- 1.5 сек. ---
dQ/Q ПЕВД+ИВД+ЭВД -0.2797211318395744
Prop:--- 0.0 сек. ---
Iter:--- 0.4 сек. ---
Else:--- 0.0 сек. 

In [2]:
#Вывод газовых потоков
gas_streams

,T,P,H,G,N2,O2,CO2,H2O,Ar
GTU-PEVD,542.100000,0.1,958.637030,503.8,0.7803,0.1237,0.0301,0.0594,0.0065
PEVD-IVD,467.045980,0.1,872.894580,503.8,0.7803,0.1237,0.0301,0.0594,0.0065
IVD-EVD,299.351383,0.1,686.682430,503.8,0.7803,0.1237,0.0301,0.0594,0.0065
EVD-PPND,231.106329,0.1,612.933342,503.8,0.7803,0.1237,0.0301,0.0594,0.0065
PPND-IND,228.410243,0.1,610.041593,503.8,0.7803,0.1237,0.0301,0.0594,0.0065
IND-GPK,174.800483,0.1,552.851926,503.8,0.7803,0.1237,0.0301,0.0594,0.0065
GPK-out,107.617416,0.1,481.909310,503.8,0.7803,0.1237,0.0301,0.0594,0.0065
NaN,NaN,0.1,NaN,503.8,0.7803,0.1237,0.0301,0.0594,0.0065


In [3]:
#Вывод водяных потоков
water_streams

,T,P,H,G,S
PEVD-DROSVD,510.014177,8.404896,3419.548097,63.596701,NaN
IVD-PEVD,301.297556,8.746000,2747.104023,63.596701,NaN
EVD-IVD,289.586283,8.746000,1286.702717,63.596701,NaN
PEN-EVD,166.221575,8.746000,707.115426,63.596701,NaN
BND-PEN,164.738361,0.696400,696.095567,63.596701,NaN
PPND-DROSND,208.268557,0.691403,2864.115173,14.198056,NaN
IND-PPND,164.738361,0.696400,2762.534980,14.198056,NaN
GPK-IND,164.738361,0.696400,706.579572,77.794757,NaN
X-GPK,60.000000,0.696400,251.748396,77.794757,NaN
SP1-OD,74.770000,0.038227,NaN,75.875000,NaN


In [4]:
#Невязка по всем параметрам
Qgas = KPD*gas_streams.at['GTU-PEVD','G']*(gas_streams.at['GTU-PEVD','H']-gas_streams.at['PEVD-IVD','H'])
Qwat = water_streams.at['PEVD-DROSVD','G']*(water_streams.at['PEVD-DROSVD','H']-water_streams.at['IVD-PEVD','H'])
print('dQ/Q ПЕВД+ИВД',((Qgas-Qwat)/Qgas*100))

Qgas1 = KPD*gas_streams.at['GTU-PEVD','G']*(gas_streams.at['GTU-PEVD','H']-gas_streams.at['IVD-EVD','H'])
Qwat1 = water_streams.at['PEVD-DROSVD','G']*(water_streams.at['PEVD-DROSVD','H']-water_streams.at['EVD-IVD','H'])
print('dQ/Q ПЕВД+ИВД+ЭВД',((Qgas1-Qwat1)/Qgas1*100))

Qgas = KPD*gas_streams.at['EVD-PPND','G']*(gas_streams.at['EVD-PPND','H']-gas_streams.at['IND-GPK','H'])
Qwat = water_streams.at['IND-PPND','G']*(water_streams.at['IND-PPND','H']-water_streams.at['GPK-IND','H'])+water_streams.at['PPND-DROSND','G']*(water_streams.at['PPND-DROSND','H']-water_streams.at['IND-PPND','H'])+water_streams.at['BND-PEN','G']*(water_streams.at['BND-PEN','H']-water_streams.at['GPK-IND','H'])
print('dQ/Q ППНД+ИНД',((Qgas-Qwat)/Qgas*100))

Qgas1 = KPD*gas_streams.at['EVD-PPND','G']*(gas_streams.at['EVD-PPND','H']-gas_streams.at['GPK-out','H'])
Qwat1 = water_streams.at['GPK-IND','G']*(water_streams.at['GPK-IND','H']-water_streams.at['X-GPK','H'])+water_streams.at['IND-PPND','G']*(water_streams.at['IND-PPND','H']-water_streams.at['GPK-IND','H'])+water_streams.at['PPND-DROSND','G']*(water_streams.at['PPND-DROSND','H']-water_streams.at['IND-PPND','H'])+water_streams.at['BND-PEN','G']*(water_streams.at['BND-PEN','H']-water_streams.at['GPK-IND','H'])
print('dQ/Q ППНД+ИНД+ГПК',(Qgas1-Qwat1)/Qgas1*100)

Qgasall = KPD*gas_streams.at['GTU-PEVD','G']*(gas_streams.at['GTU-PEVD','H']-gas_streams.at['GPK-out','H'])
Qwatall = water_streams.at['PPND-DROSND','G']*(water_streams.at['PPND-DROSND','H']-water_streams.at['X-GPK','H'])+water_streams.at['PEVD-DROSVD','G']*(water_streams.at['PEVD-DROSVD','H']-water_streams.at['X-GPK','H'])-water_streams.at['BND-PEN','G']*(water_streams.at['PEN-EVD','H']-water_streams.at['BND-PEN','H'])
print('dQ/Qsumm',(Qgasall-Qwatall)/Qgasall*100)

dQ/Q ПЕВД+ИВД -0.00034812459259379446
dQ/Q ПЕВД+ИВД+ЭВД -0.0009655154415562669
dQ/Q ППНД+ИНД 0.0008782711614091305
dQ/Q ППНД+ИНД+ГПК 0.0004027335627185439
dQ/Qsumm -0.03265448647471265


In [5]:
gas_streams0

,T,P,H,G,N2,O2,CO2,H2O,Ar
GTU-PEVD,542.10,0.1,958.637030,503.8,0.7803,0.1237,0.0301,0.0594,0.0065
PEVD-IVD,468.05,0.1,874.031830,503.8,0.7803,0.1237,0.0301,0.0594,0.0065
IVD-EVD,309.20,0.1,697.417326,503.8,0.7803,0.1237,0.0301,0.0594,0.0065
EVD-PPND,235.90,0.1,618.078812,503.8,0.7803,0.1237,0.0301,0.0594,0.0065
PPND-IND,233.00,0.1,614.965387,503.8,0.7803,0.1237,0.0301,0.0594,0.0065
IND-GPK,175.50,0.1,553.594555,503.8,0.7803,0.1237,0.0301,0.0594,0.0065
GPK-out,106.90,0.1,481.155410,503.8,0.7803,0.1237,0.0301,0.0594,0.0065
NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
electric

,N,KPD
PEN,NaN,NaN
CVD,NaN,NaN
CND,NaN,NaN


In [7]:
heaters

,Qw,Qg,KPD
PEVD,NaN,NaN,NaN
IVD,NaN,NaN,NaN
EVD,NaN,NaN,NaN
PPND,NaN,NaN,NaN
IND,NaN,NaN,NaN
GPK,NaN,NaN,NaN
